In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

import sys
sys.path.append('/home/chendian/PURE')

import time
import logging
from glob import glob
from pure_api import PureApi
from shared.data_structures import Dataset

use_cuda: True


In [50]:
# load test dataset
task_name = 'msra'
# task_name = 'resume'
# task_name = 'onto4'
test_data = os.path.join(f'../data/{task_name.replace("msra", "msra_origin")}/', f'test.with_span_score.json')
documents = Dataset(test_data)

In [51]:
# model candidates
model_dir = '/data/chendian/pure_output_dir'
# model_dir = '../output_dir'
boundary_only_mode = 'span'
span_filter_method = 'rate'
file_name_pattern = f'{model_dir}/{task_name.replace("msra", "msra_origin")}_{boundary_only_mode}{span_filter_method[0]}_*e*_wn/'
print(file_name_pattern)
thres_case = [_p.split("_")[-2] for _p in glob(file_name_pattern)]
threshold_case = sorted(thres_case, key=lambda x: float(x))
# del threshold_case[1:4]
threshold_case

/data/chendian/pure_output_dir/msra_origin_spanr_*e*_wn/


['0e-0',
 '5e-3',
 '1e-2',
 '5e-2',
 '1e-1',
 '2e-1',
 '3e-1',
 '4e-1',
 '5e-1',
 '6e-1',
 '7e-1',
 '8e-1',
 '9e-1',
 '1e-0']

In [11]:
# !mv ../output_dir/msra_origin_spanf_*_wn /data/chendian/pure_output_dir/

In [52]:
# predict on different models with different threshold
rec = {}
logger = logging.getLogger('root')
logger.setLevel(logging.WARN)


for boundary_only_mode in ['span']:  # 'bin', 'bdy', 
    cur_args = {
        'task': task_name,
        'max_span_length': 40 if task_name == 'findoc' else 25,
        'boundary_token': 'both',
        'boundary_det_filter_method': 'max',
        'boundary_only_mode': f"{boundary_only_mode}_score",
        'span_filter_method': f"{span_filter_method}",
        'eval_batch_size': 16,
    }

    # for trn_threshold in threshold_case:
    for trn_threshold in ['5e-1']:
        load_model_dir = f'{model_dir}/{task_name.replace("msra", "msra_origin")}_{boundary_only_mode}{span_filter_method[0]}_{str(trn_threshold)}_wn/'
        # for load_model_dir in glob(f'{model_dir}/{task_name}_{boundary_only_mode}f_*_wn/'):
        cur_args.update({
            "boundary_det_threshold": float(trn_threshold),
            'span_filter_method': f"{span_filter_method}",
        })
        pa = PureApi(args=cur_args, 
                     load_model_dir=load_model_dir)
        for phase in ['test']:  # 'train', 'dev', 
            st = time.time()
            for tst_threshold in ['f_0e-0', 'f_5e-1', 'f_1e-0']:  # thres
                if (trn_threshold, tst_threshold) in rec:
                    continue
                pa.args.update({
                    "boundary_det_threshold": float(tst_threshold.split('_')[1]),
                    'span_filter_method': 'thres',
                })
                p, r, f1 = pa.evaluate(documents=documents)
                rec[(trn_threshold, tst_threshold)] = (p, r, f1)
                print("cost {:.2f} seconds.".format(time.time()-st))
            for tst_threshold in ['r_1e-1', 'r_2e-1', 'r_5e-1', 'r_1e-0']:  # rate
                if (trn_threshold, tst_threshold) in rec:
                    continue
                pa.args.update({
                    "boundary_det_threshold": float(tst_threshold.split('_')[1]),
                    'span_filter_method': 'rate',
                })
                p, r, f1 = pa.evaluate(documents=documents)
                rec[(trn_threshold, tst_threshold)] = (p, r, f1)
                print("cost {:.2f} seconds.".format(time.time()-st))
        print("")
        

11/10/2021 17:25:44 - INFO - transformers.tokenization_utils_base - Model name '/data/chendian/pure_output_dir/msra_origin_spanr_5e-1_wn//' not found in model shortcut name list (bert-base-uncased, bert-large-uncased, bert-base-cased, bert-large-cased, bert-base-multilingual-uncased, bert-base-multilingual-cased, bert-base-chinese, bert-base-german-cased, bert-large-uncased-whole-word-masking, bert-large-cased-whole-word-masking, bert-large-uncased-whole-word-masking-finetuned-squad, bert-large-cased-whole-word-masking-finetuned-squad, bert-base-cased-finetuned-mrpc, bert-base-german-dbmdz-cased, bert-base-german-dbmdz-uncased, TurkuNLP/bert-base-finnish-cased-v1, TurkuNLP/bert-base-finnish-uncased-v1, wietsedv/bert-base-dutch-cased). Assuming '/data/chendian/pure_output_dir/msra_origin_spanr_5e-1_wn//' is a path, a model identifier, or url to a directory containing tokenizer files.
11/10/2021 17:25:44 - INFO - transformers.tokenization_utils_base - Didn't find file /data/chendian/pure

Take ce as loss function.


11/10/2021 17:25:48 - INFO - transformers.modeling_utils - All model checkpoint weights were used when initializing BertForEntity.

11/10/2021 17:25:48 - INFO - transformers.modeling_utils - All the weights of BertForEntity were initialized from the model checkpoint at /data/chendian/pure_output_dir/msra_origin_spanr_5e-1_wn//.
If your task is similar to the task the model of the ckeckpoint was trained on, you can already use BertForEntity for predictions without further training.


cost 144.63 seconds.
cost 181.19 seconds.
cost 206.13 seconds.
cost 238.12 seconds.
cost 280.71 seconds.
cost 407.69 seconds.



In [53]:
# dump the prediction results
import pickle
# pickle.dump(rec, open(f'../data/filtering_methods/heatmap_on_models_with_{span_filter_method}_{task_name}_211110.pkl', 'wb'))
pickle.dump(rec, open(f'../data/filtering_methods/model_with_{span_filter_method}_{task_name}_compare_211110.pkl', 'wb'))
# rec = pickle.load(open(f'../data/heatmap_on_models_with_rate_{task_name}_211022.pkl', 'rb'))

In [54]:
len(rec)

6

In [55]:
!ls ../data/filtering_methods/

heatmap_on_models_with_counts_onto4_211109.pkl
heatmap_on_models_with_rate_msra_211022.pkl
heatmap_on_models_with_rate_msra_211104.pkl
heatmap_on_models_with_rate_onto4_211109.pkl
heatmap_on_models_with_rate_resume_211104.pkl
heatmap_on_models_with_span_filtering_211003.pkl
heatmap_on_models_with_thres_msra_211023.pkl
heatmap_on_models_with_thres_on_0e-0_211020.pkl
heatmap_on_models_with_thres_on_0e-0_msra_211020.pkl
heatmap_on_models_with_thres_on_0e-0_onto4_211020.pkl
heatmap_on_models_with_thres_on_0e-0_resume_211020.pkl
heatmap_on_models_with_thres_on_5e-1_211020.pkl
model_with_rate_msra_compare_211110.pkl
model_with_rate_onto4_compare_211110.pkl
model_with_rate_resume_compare_211110.pkl


In [40]:
# show the heatmap 
print("Onto4 Dataset:\n") 
print('\t\t  TakeAll \t\t Ours(20%) \t\t Ours(10%) \t\t Pipeline') 
for trn_threshold in ['5e-1']: 
    print(f"{trn_threshold}", end='\t')
    for tst_threshold in ['f_0e-0', 'r_2e-1', 'r_1e-1', 'f_5e-1']: 
        p, r, f = rec[(trn_threshold, tst_threshold)] 
        p, r, f = 100.*p, 100.*r, 100.*f 
        # print(f'{p:.2f}/{r:.2f}/{f:.2f}', end='\t') 
        print(f'{p:.2f}/{r:.2f}/{f:.2f}', end='\t') 
    print("") 

Onto4 Dataset:

		  TakeAll 			 Ours(20%) 			 Ours(10%) 			 Pipeline
5e-1	83.31/80.01/81.63	5e-1	83.31/80.01/81.63	5e-1	83.30/80.00/81.62	5e-1	85.39/77.58/81.30	


In [49]:
# show the heatmap 
print("Resume Dataset:\n") 
print('\t\t  TakeAll \t\t Ours(20%) \t\t Ours(10%) \t\t Pipeline') 
for trn_threshold in ['5e-1']: 
    print(f"{trn_threshold}", end='\t')
    for tst_threshold in ['f_0e-0', 'r_2e-1', 'r_1e-1', 'f_5e-1']: 
        p, r, f = rec[(trn_threshold, tst_threshold)] 
        p, r, f = 100.*p, 100.*r, 100.*f 
        # print(f'{p:.2f}/{r:.2f}/{f:.2f}', end='\t') 
        print(f'{p:.2f}/{r:.2f}/{f:.2f}', end='\t') 
    print("") 

Resume Dataset:

		  TakeAll 			 Ours(20%) 			 Ours(10%) 			 Pipeline
5e-1	95.96/96.07/96.01	5e-1	95.95/95.89/95.92	5e-1	95.92/95.15/95.53	5e-1	96.70/95.28/95.98	


In [61]:
# show the heatmap 
print("MSRA Dataset:\n") 
print('\t\t  TakeAll \t\t Ours(50%) \t\t Ours(20%) \t\t Ours(10%) \t\t Pipeline') 
for trn_threshold in ['5e-1']: 
    print(f"{trn_threshold}", end='\t')
    for tst_threshold in ['f_0e-0', 'r_5e-1', 'r_2e-1', 'r_1e-1', 'f_5e-1']: 
        p, r, f = rec[(trn_threshold, tst_threshold)] 
        p, r, f = 100.*p, 100.*r, 100.*f 
        # print(f'{p:.2f}/{r:.2f}/{f:.2f}', end='\t') 
        print(f'{p:.2f}/{r:.2f}/{f:.2f}', end='\t') 
    print("") 

MSRA Dataset:

		  TakeAll 		 Ours(20%) 		 Ours(10%) 		 Pipeline
5e-1	96.05/95.75/95.90	96.05/95.75/95.90	96.06/95.75/95.91	97.25/94.20/95.70	


In [7]:
len(threshold_case)

11